In [1]:
from pydbsp.zset import ZSet, ZSetAddition
from pydbsp.stream import step_until_fixpoint
from typing import Tuple, List

from pydbsp.algorithms.datalog import (
    EDB,
    Fact,
    Program,
    Rule,
)

Edge = Tuple[int, int]
GraphZSet = ZSet[Edge]

def create_test_zset_graph(n: int) -> GraphZSet:
    return ZSet({(k, k + 1): 1 for k in range(n)})

def create_test_edb(n: int) -> EDB:
    test_graph = create_test_zset_graph(n)
    group: ZSetAddition[Fact] = ZSetAddition()

    test_edb = group.identity()
    for k, v in test_graph.items():
        test_edb.inner[("E", (k[0], k[1]))] = v

    return test_edb

def from_facts_into_zset(facts: List[Tuple[Fact, int]]) -> EDB:
    edb: EDB = ZSetAddition().identity()
    for fact, weight in facts:
        edb.inner[fact] = weight

    return edb


def from_rule_into_zset(rule: Rule, weight: int) -> Program:
    program: Program = ZSetAddition().identity()
    program.inner[rule] = weight

    return program

from pydbsp.algorithms.datalog import IncrementalDatalog, Variable
from pydbsp.stream import Stream, StreamHandle
from pydbsp.stream.operators.linear import stream_elimination

edb_group: ZSetAddition[Fact] = ZSetAddition()
edb_stream = Stream(edb_group)
edb_stream_h = StreamHandle(lambda: edb_stream)

program_group: ZSetAddition[Tuple[int, Rule]]= ZSetAddition()
program_stream = Stream(program_group)
program_stream_h = StreamHandle(lambda: program_stream)

seed: Rule = (("T", (Variable("X"), Variable("Y"))), ("E", (Variable("X"), Variable("Y"))))
transitivity: Rule = (
    ("T", (Variable("X"), Variable("Z"))),
    ("T", (Variable("X"), Variable("Y"))),
    ("T", (Variable("Y"), Variable("Z"))),
)

class FromProgramIntoGraph(): ...
# For each rule...
# # For each body atom
# # # ( rule.predicate, hash (rule with body atom in head) )

class PropagateCore(): ...

reasoner = IncrementalDatalog(edb_stream_h, program_stream_h, None)
edb_stream.send(from_facts_into_zset([(("E", (2, 3)), 1), (("E", (0, 2)), 1), (("E", (0, 1)), 1), (("E", (3, 4)), 1)]))
program_stream.send(from_rule_into_zset(transitivity, 1))
program_stream.send(from_rule_into_zset(seed, 1))

step_until_fixpoint(reasoner)
print(stream_elimination(reasoner.output()))

{('E', (2, 3)): 1, ('E', (0, 2)): 1, ('E', (0, 1)): 1, ('E', (3, 4)): 1, ('T', (2, 3)): 1, ('T', (0, 2)): 1, ('T', (0, 1)): 1, ('T', (3, 4)): 1, ('T', (2, 4)): 1, ('T', (0, 3)): 1, ('T', (0, 4)): 1}
